## Task 1: landcover classification

### Retrieve S2 embeddings from an S3 bucket over California

In [1]:
import geopandas as gpd
import s3fs
import pandas as pd
import os

from tqdm import tqdm


file_path = 'models/land_cover_model.h5'

if os.path.exists(file_path):
    print("Model already exists")
else:
    # S3 bucket and prefix
    s3_bucket = 'clay-worker-bucket-dev-small-tasks'
    s3_prefix = '_data/gpq/87/'#51/' #2022 S2 Cali embeddings

    # Initialize s3fs filesystem
    fs = s3fs.S3FileSystem()

    # List all the GeoParquet files in the specified S3 directory
    geo_parquet_files = fs.glob(f's3://{s3_bucket}/{s3_prefix}*.gpq')

    # List to store GeoDataFrames
    gdfs = []

    # Load each GeoParquet file into a GeoDataFrame and append to the list
    for file in tqdm(geo_parquet_files):
        gdf = gpd.read_parquet("s3://"+file, storage_options={"anon": False, "client_kwargs": {"region_name": "us-west-2"}})
        gdfs.append(gdf)

    # Concatenate all GeoDataFrames into a single GeoDataFrame
    combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

    # Display the concatenated GeoDataFrame
    combined_gdf


Model already exists


### Join most common land use value that corresponds to each chip

In [2]:
import ee

from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

if os.path.exists(file_path):
    print("Model already exists")
else:
    # Initialize Earth Engine
    ee.Initialize()

    # Load the ESRI Global LULC dataset
    lulc = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS").mosaic()

    def get_most_common_lulc(geometry):
        # Convert the GeoPandas geometry to an Earth Engine geometry
        ee_geometry = ee.Geometry.Rectangle(geometry.bounds)
        
        # Get the LULC values within the bounding box
        lulc_values = lulc.reduceRegion(
            reducer=ee.Reducer.frequencyHistogram(),
            geometry=ee_geometry,
            scale=10,
            maxPixels=1e9
        ).get('b1')
        
        # Find the most common LULC value
        lulc_dict = ee.Dictionary(lulc_values)
        most_common = lulc_dict.keys().sort(lulc_dict.values()).get(-1)
        
        # Return the result
        return int(most_common.getInfo())

    def process_geometries(combined_gdf):
        # Get the total number of rows in the GeoDataFrame
        total_rows = len(combined_gdf)

        # Determine the number of threads to use
        max_threads = 10  # Adjust this based on your system and Earth Engine quota
        num_threads = min(total_rows, max_threads)

        results = []

        with ThreadPoolExecutor(max_workers=num_threads) as executor:
            # Submit all tasks
            future_to_index = {executor.submit(get_most_common_lulc, row.geometry): index 
                            for index, row in combined_gdf.iterrows()}
            
            # Process as they complete with a progress bar
            with tqdm(total=total_rows, desc="Processing bounding boxes") as pbar:
                for future in as_completed(future_to_index):
                    index = future_to_index[future]
                    try:
                        result = future.result()
                    except Exception as exc:
                        print(f'Generated an exception: {exc}')
                        result = None
                    results.append((index, result))
                    pbar.update(1)

        # Sort results by index and extract only the values
        sorted_results = [r[1] for r in sorted(results, key=lambda x: x[0])]
        
        return sorted_results

    if __name__ == '__main__':
        # Process the geometries
        results = process_geometries(combined_gdf)

        # Add the results as a new column to the GeoDataFrame
        combined_gdf['most_common_lulc'] = results

        # Save as GeoJSON
        combined_gdf[['geometry', 'most_common_lulc']].to_file("test_lulc.geojson", driver="GeoJSON")

Model already exists


### Tune hyperparameters for land use classifier using Optuna

In [3]:
import pandas as pd
import numpy as np
import optuna
import tensorflow as tf
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.optimizers import Adam
from keras.utils import to_categorical

class NeuralNetwork:
    def __init__(self, input_shape, layers, dropout_rate, learning_rate, num_classes, device):
        self.input_shape = input_shape
        self.layers = layers
        self.dropout_rate = dropout_rate
        self.learning_rate = learning_rate
        self.num_classes = num_classes
        self.device = device
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Input(shape=(self.input_shape,)))
        for layer_size in self.layers:
            model.add(Dense(layer_size, activation='relu'))
            model.add(Dropout(self.dropout_rate))
        model.add(Dense(self.num_classes, activation='softmax'))
        return model

    def compile_model(self):
        optimizer = Adam(learning_rate=self.learning_rate)
        self.model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    def train_model(self, X_train, y_train, epochs=20, batch_size=32, validation_split=0.2):
        with tf.device(self.device):
            history = self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
        return history

    def evaluate_model(self, X_test, y_test):
        with tf.device(self.device):
            loss, accuracy = self.model.evaluate(X_test, y_test)
        return loss, accuracy

    def predict(self, X_test):
        with tf.device(self.device):
            predictions = self.model.predict(X_test)
        return np.argmax(predictions, axis=1)

    def calculate_f1(self, y_test, predictions):
        return f1_score(np.argmax(y_test, axis=1), predictions, average='weighted')
    
    def save_model(self, filename):
        self.model.save(filename)

    @classmethod
    def load_model(cls, filename, input_shape, num_classes, device):
        loaded_model = tf.keras.models.load_model(filename)
        nn = cls(input_shape, [], 0, 0, num_classes, device)  # Dummy values for layers, dropout_rate, and learning_rate
        nn.model = loaded_model
        return nn


def objective(trial):
    layers = []
    for i in range(trial.suggest_int('n_layers', 1, 3)):
        layers.append(trial.suggest_int(f'n_units_l{i}', 64, 512))
    
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)
    
    nn = NeuralNetwork(input_shape=X_train.shape[1], device=device, layers=layers, dropout_rate=dropout_rate, learning_rate=learning_rate, num_classes=num_classes)
    nn.compile_model()
    
    nn.train_model(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)
    
    predictions = nn.predict(X_test)
    f1 = nn.calculate_f1(y_test, predictions)
    
    return f1

if os.path.exists(file_path):
    print("Model already exists")
else:
    # Assuming combined_gdf is already loaded
    X = combined_gdf['embeddings'].tolist()
    X = pd.DataFrame(X)
    y = combined_gdf['most_common_lulc']

    # Encode the target variable
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    num_classes = len(label_encoder.classes_)

    # Convert to one-hot encoding
    y_onehot = to_categorical(y_encoded)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

    # Scale the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'

    # Optimize the hyperparameters
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=20)

    # Print the best hyperparameters
    print(study.best_params)

    # Example usage with the best hyperparameters
    best_params = study.best_params
    layers = [best_params[f'n_units_l{i}'] for i in range(best_params['n_layers'])]
    dropout_rate = best_params['dropout_rate']
    learning_rate = best_params['learning_rate']

    nn = NeuralNetwork(input_shape=X_train.shape[1], device=device, layers=layers, dropout_rate=dropout_rate, learning_rate=learning_rate, num_classes=num_classes)
    nn.compile_model()
    history = nn.train_model(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)
    loss, accuracy = nn.evaluate_model(X_test, y_test)
    print(f'Test Accuracy: {accuracy:.4f}')
    predictions = nn.predict(X_test)
    f1 = nn.calculate_f1(y_test, predictions)
    print(f'Test F1 Score: {f1:.4f}')

    # Save the model
    nn.save_model('models/land_cover_model.h5')

    # Save scaler and label_encoder
    joblib.dump(scaler, 'models/scaler.joblib')
    joblib.dump(label_encoder, 'models/label_encoder.joblib')

    # If you want to get the actual class labels
    predicted_classes = label_encoder.inverse_transform(predictions)

/opt/conda/envs/claymodel-latest-v2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-27 23:24:04.899295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-27 23:24:04.917480: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-27 23:24:04.923098: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-27 23:24:04.935962: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlo

Model already exists


### Predict on test set

#### Load in test set geojson and turn into grid of points

In [4]:
import geopandas as gpd

year = 2022

# Load the GeoJSON file
geojson_path = 'test_data/challenge_1_bb.geojson'
gdf = gpd.read_file(geojson_path)
gdf

,geometry
0,"POLYGON ((-106.08092 35.78627, -106.08092 35.4..."


In [5]:
import pyproj

def get_utm_zone(longitude):
    return int((longitude + 180) / 6) + 1

# Get the bounds of the geometry
minx, miny, maxx, maxy = gdf.geometry.bounds.iloc[0]

# Calculate UTM zone
utm_zone = get_utm_zone(minx)

# Check for a suitable projection using pyproj
proj = pyproj.Proj(proj='utm', zone=utm_zone, ellps='WGS84')

# Get the corresponding EPSG code for the UTM zone using pyproj
utm_crs = pyproj.CRS(f"+proj=utm +zone={utm_zone} +datum=WGS84")
epsg_code = utm_crs.to_epsg()

# Reproject the GeoDataFrame to the chosen EPSG code
gdf = gdf.to_crs(epsg=epsg_code)
gdf

,geometry
0,"POLYGON ((402315.263 3960781.699, 401878.759 3..."


In [6]:
import numpy as np

# Create a grid of points 5120m apart
x = np.arange(gdf.total_bounds[0], gdf.total_bounds[2], 2560)
y = np.arange(gdf.total_bounds[1], gdf.total_bounds[3], 2560)
xx, yy = np.meshgrid(x, y)
points = np.vstack([xx.ravel(), yy.ravel()]).T

grid = gpd.GeoDataFrame(geometry=gpd.points_from_xy(points[:, 0], points[:, 1], crs=gdf.crs))
grid

,geometry
0,POINT (401878.759 3920624.607)
1,POINT (404438.759 3920624.607)
2,POINT (406998.759 3920624.607)
3,POINT (409558.759 3920624.607)
4,POINT (412118.759 3920624.607)
...,...
331,POINT (442838.759 3959024.607)
332,POINT (445398.759 3959024.607)
333,POINT (447958.759 3959024.607)
334,POINT (450518.759 3959024.607)


#### Download S2 data for the grid defined above

In [7]:
import stackstac
import rasterio
import pystac_client
import rioxarray

from shapely.geometry import Point

def download_stack_images(start_date, end_date, grid, output_directory):
    os.makedirs(output_directory, exist_ok=True)
    
    # Band groups updated with new bands
    BAND_GROUPS = {
        "rgb": ["red", "green", "blue"],
        "rededge": ["rededge1", "rededge2", "rededge3", "nir08"],
        "nir": ["nir"],
        "swir": ["swir16", "swir22"],
        "sar": ["vv", "vh"],
    }

    # STAC API and Collection details
    STAC_API = "https://earth-search.aws.element84.com/v1"
    COLLECTION = "sentinel-2-l2a"

    # Initialize STAC client
    catalog = pystac_client.Client.open(STAC_API)

    # Points of Interest as a list of tuples (longitude, latitude)
    points = grid.to_crs("EPSG:4326").geometry.apply(lambda x: (x.x, x.y)).tolist()

    # Iterate over each point of interest
    cnt = 0
    indices_for_join = []
    for lon, lat in tqdm(points):
        bbox = [lon - 1e-5, lat - 1e-5, lon + 1e-5, lat + 1e-5]

        # Search for items with low cloud cover, without sorting by cloud cover
        search = catalog.search(
            collections=[COLLECTION],
            datetime=f"{start_date}/{end_date}",
            bbox=bbox,
            max_items=10,
            query={"eo:cloud_cover": {"lt": 20}}  # Assuming 'eo:cloud_cover' is valid for filtering
        )

        items = list(search.get_items())
        if not items:
            print("No items found with low cloud cover.")
            continue

        # Optionally, sort items manually by cloud cover if it's available in the properties
        items = sorted(items, key=lambda x: x.properties.get('eo:cloud_cover', float('inf')))

        # Select the item with the lowest cloud cover
        lowest_cloud_item = items[0]

        # Convert point into the image projection of the selected item
        epsg = lowest_cloud_item.properties["proj:epsg"]
        point_gdf = gpd.GeoDataFrame(
            [{'geometry': Point(lon, lat)}], 
            crs='EPSG:4326'
        ).to_crs(epsg=epsg)
        coords = list(point_gdf.iloc[0].geometry.coords)[0]

        bounds = (
            coords[0] - 1120, coords[1] - 1120,
            coords[0] + 1120, coords[1] + 1120,
        )

        # Process and visualize the selected image
        stack = stackstac.stack(
            [lowest_cloud_item],
            bounds=bounds,
            snap_bounds=False,
            epsg=epsg,
            resolution=10,
            dtype="float32",
            rescale=False,
            fill_value=np.nan,
            assets=BAND_GROUPS["rgb"] + BAND_GROUPS["nir"],  # TODO: Add more/all of the Sentinel 2 bands to see if embeddings improve
        ).compute()

        # Assuming 'stack' is your xarray DataArray loaded with stackstac
        stack = stack.rio.write_crs("EPSG:4326")  # You can change to your specific EPSG code

        mgrs = str(stack.coords["grid:code"].values).split("-")[1]
        date = str(stack.time.values)[2:11]

        output_path = os.path.join(output_directory, f"stack_{lon}_{lat}_{cnt}.tif")
        indices_for_join.append(cnt)

        if 'time' in stack.dims:
            stack = stack.isel(time=0)

        # Write the stack to a TIFF file
        with rasterio.open(
                output_path, 'w',
                driver='GTiff',
                height=stack.shape[1],
                width=stack.shape[2],
                count=len(stack.band),  # Number of bands
                dtype=str(stack.dtype),
                crs=epsg, # TODO: Confirm this
                transform=stack.rio.transform()
            ) as tif:
            for i, band in enumerate(stack.band, start=1):
                tif.write(stack.sel(band=band).values, i)

        # Reopen the file to add metadata
        with rasterio.open(output_path, "r+") as rst:
            rst.update_tags(date=date)

        cnt += 1

    grid["join_index"] = indices_for_join

In [8]:
download_stack_images(f"{year}-01-01", f"{year}-12-31", grid, "test_data/embeddings/challenge_1")

  0%|          | 0/336 [00:00<?, ?it/s]/opt/conda/envs/claymodel-latest-v2/lib/python3.11/site-packages/pystac_client/item_search.py:836: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(
/opt/conda/envs/claymodel-latest-v2/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(
  0%|          | 1/336 [00:01<09:54,  1.77s/it]/opt/conda/envs/claymodel-latest-v2/lib/python3.11/site-packages/pystac_client/item_search.py:836: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(
/opt/conda/envs/claymodel-latest-v2/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in 

#### Join LULC data on grid

In [9]:
# Initialize Earth Engine
ee.Initialize()

# Load the ESRI Global LULC dataset
lulc = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS").mosaic()

def get_most_common_lulc(geometry):
    # Convert the GeoPandas geometry to an Earth Engine geometry
    ee_geometry = ee.Geometry.Rectangle(geometry.bounds)
    
    # Get the LULC values within the bounding box
    lulc_values = lulc.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=ee_geometry,
        scale=10,
        maxPixels=1e9
    ).get('b1')
    
    # Find the most common LULC value
    lulc_dict = ee.Dictionary(lulc_values)
    most_common = lulc_dict.keys().sort(lulc_dict.values()).get(-1)
    
    # Return the result
    return int(most_common.getInfo())

def process_geometries(combined_gdf):
    # Get the total number of rows in the GeoDataFrame
    total_rows = len(combined_gdf)

    # Determine the number of threads to use
    max_threads = 10  # Adjust this based on your system and Earth Engine quota
    num_threads = min(total_rows, max_threads)

    results = []

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Submit all tasks
        future_to_index = {executor.submit(get_most_common_lulc, row.geometry): index 
                        for index, row in combined_gdf.iterrows()}
        
        # Process as they complete with a progress bar
        with tqdm(total=total_rows, desc="Processing bounding boxes") as pbar:
            for future in as_completed(future_to_index):
                index = future_to_index[future]
                try:
                    result = future.result()
                except Exception as exc:
                    print(f'Generated an exception: {exc}')
                    result = None
                results.append((index, result))
                pbar.update(1)

    # Sort results by index and extract only the values
    sorted_results = [r[1] for r in sorted(results, key=lambda x: x[0])]
    
    return sorted_results

if __name__ == '__main__':
    # Process the geometries
    results = process_geometries(grid.to_crs("EPSG:4326"))

    # Add the results as a new column to the GeoDataFrame
    grid['most_common_lulc'] = results

    # Save as GeoJSON
    grid[['geometry', 'most_common_lulc']].to_file("test_lulc_v2.geojson", driver="GeoJSON")

Processing bounding boxes: 100%|██████████| 336/336 [00:06<00:00, 54.88it/s]


#### Generate embeddings for the images

In [10]:
import os
import pystac_client
import numpy as np
import requests
import yaml
import math
import torch
import torchvision.transforms as v2
import geopandas as gpd
from shapely.geometry import Point
from box import Box
from stacchip.indexer import Sentinel2Indexer
from stacchip.chipper import Chipper
from tqdm import tqdm

# Optimize GDAL settings for cloud optimized reading
os.environ["GDAL_DISABLE_READDIR_ON_OPEN"] = "EMPTY_DIR"
os.environ["AWS_REQUEST_PAYER"] = "requester"

STAC_API = "https://earth-search.aws.element84.com/v1"
COLLECTION = "sentinel-2-l2a"

# Initialize lists to store results
all_cls_embeddings = []
geometries = []
# Points of Interest as a list of tuples (longitude, latitude)
points = grid.to_crs("EPSG:4326").geometry.apply(lambda x: (x.x, x.y)).tolist()

torch.cuda.empty_cache()

clay_encoder = torch.export.load("clay-v1-encoder.pt").module()

for lon, lat in tqdm(points):
    # Search the catalogue
    catalog = pystac_client.Client.open(STAC_API)
    search = catalog.search(
        collections=[COLLECTION],
        datetime=f"{year}-01-01/{year}-12-31",
        bbox=(lon - 1e-5, lat - 1e-5, lon + 1e-5, lat + 1e-5),
        max_items=100,
        query={"eo:cloud_cover": {"lt": 80}},
    )

    all_items = search.get_all_items()

    # Sort items by cloud cover and select the one with the lowest cloud cover
    if all_items:
        sorted_items = sorted(all_items, key=lambda x: x.properties.get('eo:cloud_cover', float('inf')))
        lowest_cloud_item = sorted_items[0]
    else:
        print(f"No items found for lat: {lat}, lon: {lon}")
        continue

    print(f"Selected item {lowest_cloud_item.id} for lat: {lat}, lon: {lon}")

    chips = []
    datetimes = []
    bboxs = []
    chip_ids = []
    item_ids = []

    print(f"Working on {lowest_cloud_item}")

    # Index the chips in the item
    indexer = Sentinel2Indexer(lowest_cloud_item)

    # Instantiate the chipper
    chipper = Chipper(indexer, assets=["red", "green", "blue", "nir", "scl"])

    # Get chips for the "image" asset key
    for idx, (x, y, chip) in enumerate(chipper):
        if idx > 2:
            break
        del chip["scl"]
        chips.append(chip)
        datetimes.append(lowest_cloud_item.datetime)
        bboxs.append(indexer.get_chip_bbox(x, y))
        chip_ids.append((x, y))
        item_ids.append(lowest_cloud_item.id)

    pixels = np.array([np.array(list(chip.values())).squeeze() for chip in chips])

    # Extract mean, std, and wavelengths from metadata
    platform = "sentinel-2-l2a"
    # Retrieve the file content from the URL

    url = (
        "https://raw.githubusercontent.com/Clay-foundation/model/main/configs/metadata.yaml"
    )
    response = requests.get(url, allow_redirects=True)

    # Convert bytes to string
    content = response.content.decode("utf-8")

    # Load the yaml
    content = yaml.safe_load(content)

    metadata = Box(content)
    mean = []
    std = []
    waves = []
    # Use the band names to get the correct values in the correct order.
    for band in chips[0].keys():
        mean.append(metadata[platform].bands.mean[band])
        std.append(metadata[platform].bands.std[band])
        waves.append(metadata[platform].bands.wavelength[band])

    # Prepare the normalization transform function using the mean and std values.
    transform = v2.Compose(
        [
            v2.Normalize(mean=mean, std=std),
        ]
    )

    def normalize_timestamp(date):
        week = date.isocalendar().week * 2 * np.pi / 52
        hour = date.hour * 2 * np.pi / 24

        return (math.sin(week), math.cos(week)), (math.sin(hour), math.cos(hour))

    times = [normalize_timestamp(dat) for dat in datetimes]
    week_norm = [dat[0] for dat in times]
    hour_norm = [dat[1] for dat in times]

    # Prep lat/lon embedding using the
    def normalize_latlon(lat, lon):
        lat = lat * np.pi / 180
        lon = lon * np.pi / 180

        return (math.sin(lat), math.cos(lat)), (math.sin(lon), math.cos(lon))

    latlons = [normalize_latlon(lat, lon)] * len(times)
    lat_norm = [dat[0] for dat in latlons]
    lon_norm = [dat[1] for dat in latlons]

    # Prep gsd
    gsd = torch.tensor([10.0], dtype=torch.float32)  # GSD should be a scalar

    # Normalize pixels
    pixels = pixels.astype(np.float32)
    pixels = torch.tensor(pixels, dtype=torch.float32)
    pixels = transform(pixels)

    # Ensure all tensors are float32 and on the same device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # device = torch.device("cpu")
    pixels = pixels.to(device=device, dtype=torch.float32)
    week_norm = torch.tensor(np.hstack((week_norm, hour_norm)), dtype=torch.float32, device=device)
    lat_norm = torch.tensor(np.hstack((lat_norm, lon_norm)), dtype=torch.float32, device=device)
    waves = torch.tensor(waves, dtype=torch.float32, device=device)
    gsd = gsd.to(device=device)

    # Process data with batch size of 1, repeating to meet minimum batch size
    batch_pixels = pixels.repeat(32, 1, 1, 1)  # Repeat to batch size 32
    batch_week_norm = week_norm.repeat(32, 1)
    batch_lat_norm = lat_norm.repeat(32, 1)

    datacube = (
        batch_pixels,
        batch_week_norm,
        batch_lat_norm,
        waves,  # Keep waves as is, no repeat
        gsd,    # GSD remains a scalar
    )

    # Ensure the model is on the same device
    clay_encoder = clay_encoder.to(device)

    # Run the clay encoder
    with torch.no_grad():
        unmsk_patch, unmsk_idx, msk_idx, msk_matrix = clay_encoder(*datacube)

    # Get class embeddings
    cls_embedding = unmsk_patch[:, 0, :]

    # Store the first embedding (batch_size 32) as the representative embedding
    all_cls_embeddings.append(cls_embedding[0].cpu())
    geometries.append(Point(lon, lat))

# Concatenate all embeddings
all_cls_embeddings_tensor = torch.stack(all_cls_embeddings, dim=0)
print(all_cls_embeddings_tensor.shape)

# Create GeoDataFrame
test_gdf = gpd.GeoDataFrame({
    'geometry': geometries,
    'embeddings': [embedding.numpy().tolist() for embedding in all_cls_embeddings_tensor]
}, crs="EPSG:4326")

test_gdf['most_common_lulc'] = grid['most_common_lulc']

# Print or save the GeoDataFrame
test_gdf.head()


/opt/conda/envs/claymodel-latest-v2/lib/python3.11/site-packages/torch/export/_unlift.py:58: UserWarning: Attempted to insert a get_attr Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule, GraphModule.add_parameter to add the necessary Parameter, or nn.Module.register_buffer to add the necessary buffer
  getattr_node = gm.graph.get_attr(lifted_node)
/opt/conda/envs/claymodel-latest-v2/lib/python3.11/site-packages/torch/fx/graph.py:1460: UserWarning: Node _lifted_tensor_constant0_1 target _lifted_tensor_constant0 _lifted_tensor_constant0 of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
  0%|          | 0/336 [00:00<?, ?it/s]/opt/conda/envs/claymodel-latest-v2/lib/python3.11/site-packages/pystac_client/item_search.py:851: FutureWarning: get_all_items() is deprecated, use ite

Selected item S2A_13SDV_20221020_0_L2A for lat: 35.42421114494862, lon: -106.0808685760685
Working on <Item id=S2A_13SDV_20221020_0_L2A>


  0%|          | 1/336 [00:10<58:15, 10.43s/it]

Selected item S2A_13SDV_20221020_0_L2A for lat: 35.42446024009015, lon: -106.05267283503353
Working on <Item id=S2A_13SDV_20221020_0_L2A>


  1%|          | 2/336 [00:18<49:40,  8.92s/it]

...

100%|█████████▉| 335/336 [38:00<00:05,  5.13s/it]

Selected item S2A_13SDV_20221020_0_L2A for lat: 35.77416335782829, lon: -105.51913053592999
Working on <Item id=S2A_13SDV_20221020_0_L2A>


100%|██████████| 336/336 [38:04<00:00,  6.80s/it]

torch.Size([336, 768])


,geometry,embeddings,most_common_lulc
0,POINT (-106.08087 35.42421),"[-0.02422812022268772, -0.08208765089511871, 0...",11
1,POINT (-106.05267 35.42446),"[-0.024211246520280838, -0.08211115002632141, ...",11
2,POINT (-106.02448 35.42470),"[-0.02419484592974186, -0.08213458210229874, 0...",11
3,POINT (-105.99628 35.42494),"[-0.02417799085378647, -0.08215818554162979, 0...",11
4,POINT (-105.96808 35.42517),"[-0.024161258712410927, -0.08218131959438324, ...",11


In [11]:
# Detect if GPU is available
device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'

# Load the model
loaded_nn = NeuralNetwork.load_model('models/land_cover_model.h5', input_shape=768, num_classes=11, device=device)

# Prepare your new data (assuming it's in the same format as your training data)
new_data = test_gdf['embeddings'].tolist()
new_data = pd.DataFrame(new_data)  # Ensure the new data is in DataFrame format

# Standardize the new data using the saved scaler
scaler = joblib.load('models/scaler.joblib')
new_data_scaled = scaler.transform(new_data)

# Make predictions
new_predictions = loaded_nn.predict(new_data_scaled)

# Load the label encoder
label_encoder = joblib.load('models/label_encoder.joblib')

# If you want to convert the predictions back to the original class labels
test_gdf['pred_lulc'] = label_encoder.inverse_transform(new_predictions)

I0000 00:00:1722125074.019869   72071 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722125074.022631   72071 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722125074.024198   72071 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722125074.038732   72071 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

 1/11 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step

I0000 00:00:1722125076.402397   72451 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 152ms/step


In [14]:
test_gdf['embeddings']

0      [-0.02422812022268772, -0.08208765089511871, 0...
1      [-0.024211246520280838, -0.08211115002632141, ...
2      [-0.02419484592974186, -0.08213458210229874, 0...
3      [-0.02417799085378647, -0.08215818554162979, 0...
4      [-0.024161258712410927, -0.08218131959438324, ...
                             ...                        
331    [-0.024046512320637703, -0.08254100382328033, ...
332    [-0.024029461666941643, -0.08256262540817261, ...
333    [-0.02401147224009037, -0.08258378505706787, 0...
334    [-0.023994266986846924, -0.08260524272918701, ...
335    [-0.023976752534508705, -0.08262651413679123, ...
Name: embeddings, Length: 336, dtype: object

In [12]:
print("Test set accuracy:", np.mean(test_gdf['pred_lulc']==test_gdf['most_common_lulc']))

Test set accuracy: 0.0


In [13]:
test_gdf['pred_lulc']

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
331    1.0
332    1.0
333    1.0
334    1.0
335    1.0
Name: pred_lulc, Length: 336, dtype: float64

In [15]:
# Ignore everything after here

#### Make predictions for the given grid

In [ ]:
import geopandas as gpd
import s3fs
import pandas as pd
import os

from tqdm import tqdm



# S3 bucket and prefix
s3_bucket = 'clay-worker-bucket-dev-small-tasks'
s3_prefix = '_data/gpq/86/' #2022 S2 embeddings over test set

# Initialize s3fs filesystem
fs = s3fs.S3FileSystem()

# List all the GeoParquet files in the specified S3 directory
geo_parquet_files = fs.glob(f's3://{s3_bucket}/{s3_prefix}*.gpq')

# List to store GeoDataFrames
gdfs = []

# Load each GeoParquet file into a GeoDataFrame and append to the list
for file in tqdm(geo_parquet_files):
    gdf = gpd.read_parquet("s3://"+file, storage_options={"anon": False, "client_kwargs": {"region_name": "us-west-2"}})
    gdfs.append(gdf)

# Concatenate all GeoDataFrames into a single GeoDataFrame
gdf_test_embed = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

# Display the concatenated GeoDataFrame
gdf_test_embed


100%|██████████| 1/1 [00:01<00:00,  1.08s/it]


,embeddings,geometry
0,"[-0.10595816, 0.049139716, -0.013616198, 0.103...","POLYGON ((-106.06225 35.59601, -106.06225 35.6..."
1,"[-0.11944625, 0.04765699, 0.0012937918, 0.0989...","POLYGON ((-106.04359 35.59601, -106.04359 35.6..."
2,"[-0.1352622, 0.03729467, 0.025111463, 0.089445...","POLYGON ((-106.06225 35.61468, -106.06225 35.6..."
3,"[-0.11955425, 0.047935113, 0.012492391, 0.0986...","POLYGON ((-106.04359 35.63335, -106.04359 35.6..."
4,"[-0.110916734, 0.043892846, 0.00046651432, 0.1...","POLYGON ((-106.04359 35.61468, -106.04359 35.6..."
...,...,...
635,"[-0.10154818, 0.04700841, -0.058602504, 0.1337...","POLYGON ((-105.48359 35.76401, -105.48359 35.7..."
636,"[-0.10063921, 0.0487299, -0.056069173, 0.12934...","POLYGON ((-105.48359 35.46535, -105.48359 35.4..."
637,"[-0.119740516, 0.050605718, -0.021452673, 0.12...","POLYGON ((-105.48359 35.78268, -105.48359 35.8..."
638,"[-0.10471448, 0.04887444, -0.05049323, 0.11969...","POLYGON ((-105.48359 35.44668, -105.48359 35.4..."


In [ ]:
# Initialize Earth Engine
ee.Initialize()

# Load the ESRI Global LULC dataset
lulc = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS").mosaic()

def get_most_common_lulc(geometry):
    # Convert the GeoPandas geometry to an Earth Engine geometry
    ee_geometry = ee.Geometry.Rectangle(geometry.bounds)
    
    # Get the LULC values within the bounding box
    lulc_values = lulc.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=ee_geometry,
        scale=10,
        maxPixels=1e9
    ).get('b1')
    
    # Find the most common LULC value
    lulc_dict = ee.Dictionary(lulc_values)
    most_common = lulc_dict.keys().sort(lulc_dict.values()).get(-1)
    
    # Return the result
    return int(most_common.getInfo())

def process_geometries(combined_gdf):
    # Get the total number of rows in the GeoDataFrame
    total_rows = len(combined_gdf)

    # Determine the number of threads to use
    max_threads = 10  # Adjust this based on your system and Earth Engine quota
    num_threads = min(total_rows, max_threads)

    results = []

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Submit all tasks
        future_to_index = {executor.submit(get_most_common_lulc, row.geometry): index 
                        for index, row in combined_gdf.iterrows()}
        
        # Process as they complete with a progress bar
        with tqdm(total=total_rows, desc="Processing bounding boxes") as pbar:
            for future in as_completed(future_to_index):
                index = future_to_index[future]
                try:
                    result = future.result()
                except Exception as exc:
                    print(f'Generated an exception: {exc}')
                    result = None
                results.append((index, result))
                pbar.update(1)

    # Sort results by index and extract only the values
    sorted_results = [r[1] for r in sorted(results, key=lambda x: x[0])]
    
    return sorted_results

if __name__ == '__main__':
    # Process the geometries
    results = process_geometries(gdf_test_embed.to_crs("EPSG:4326"))

    # Add the results as a new column to the GeoDataFrame
    gdf_test_embed['most_common_lulc'] = results

    # Save as GeoJSON
    gdf_test_embed[['geometry', 'most_common_lulc']].to_file("test_lulc_v2.geojson", driver="GeoJSON")

Processing bounding boxes: 100%|██████████| 640/640 [00:16<00:00, 38.75it/s]


In [ ]:
# Detect if GPU is available
device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'

# Load the model
loaded_nn = NeuralNetwork.load_model('models/land_cover_model.h5', input_shape=768, num_classes=11, device=device)

# Prepare your new data (assuming it's in the same format as your training data)
new_data = gdf_test_embed['embeddings'].tolist()
new_data = pd.DataFrame(new_data)  # Ensure the new data is in DataFrame format

# Standardize the new data using the saved scaler
scaler = joblib.load('models/scaler.joblib')
new_data_scaled = scaler.transform(new_data)

# Make predictions
new_predictions = loaded_nn.predict(new_data_scaled)

# Load the label encoder
label_encoder = joblib.load('models/label_encoder.joblib')

# If you want to convert the predictions back to the original class labels
gdf_test_embed['pred_lulc'] = label_encoder.inverse_transform(new_predictions)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step


In [ ]:
print("Test set accuracy:", np.mean(gdf_test_embed['pred_lulc']==gdf_test_embed['most_common_lulc']))

Test set accuracy: 0.6546875


## Task 3: Above ground stock regression

### Retrieve S2 embeddings from an S3 bucket over California

In [3]:
import geopandas as gpd
import s3fs
import pandas as pd
import os

from tqdm import tqdm


file_path = 'models/agb_regression_model.h5'

if os.path.exists(file_path):
    print("Model already exists")
else:
    # S3 bucket and prefix
    s3_bucket = 'clay-worker-bucket-dev-small-tasks'
    s3_prefix = '_data/gpq/51/' #2022 S2 Cali embeddings

    # Initialize s3fs filesystem
    fs = s3fs.S3FileSystem()

    # List all the GeoParquet files in the specified S3 directory
    geo_parquet_files = fs.glob(f's3://{s3_bucket}/{s3_prefix}*.gpq')

    # List to store GeoDataFrames
    gdfs = []

    # Load each GeoParquet file into a GeoDataFrame and append to the list
    for file in tqdm(geo_parquet_files):
        gdf = gpd.read_parquet("s3://"+file, storage_options={"anon": False, "client_kwargs": {"region_name": "us-west-2"}})
        gdfs.append(gdf)

    # Concatenate all GeoDataFrames into a single GeoDataFrame
    combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

    # Display the concatenated GeoDataFrame
    combined_gdf


Model already exists


### Join above ground biomass for each chip

In [4]:
import ee
import geopandas as gpd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

if os.path.exists(file_path):
    print("Model already exists")
    combined_gdf = gpd.read_file("agb.geojson")
else:
    # Initialize Earth Engine
    ee.Initialize()

    # Load the NASA/ORNL biomass carbon density dataset
    biomass = ee.ImageCollection("NASA/ORNL/biomass_carbon_density/v1").mosaic()

    def get_agb(geometry):
        # Convert the GeoPandas geometry to an Earth Engine geometry
        ee_geometry = ee.Geometry.Polygon(list(geometry.exterior.coords))

        # Get the mean AGB value within the geometry
        agb_value = biomass.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=ee_geometry,
            scale=300,
            maxPixels=1e9
        ).get('agb')
        
        # Return the result
        return agb_value.getInfo() if agb_value is not None else None

    def process_geometries(combined_gdf):
        # Get the total number of rows in the GeoDataFrame
        total_rows = len(combined_gdf)

        # Determine the number of threads to use
        max_threads = 10  # Adjust this based on your system and Earth Engine quota
        num_threads = min(total_rows, max_threads)

        results = []

        with ThreadPoolExecutor(max_workers=num_threads) as executor:
            # Submit all tasks
            future_to_index = {executor.submit(get_agb, row.geometry): index 
                            for index, row in combined_gdf.iterrows()}
            
            # Process as they complete with a progress bar
            with tqdm(total=total_rows, desc="Processing geometries") as pbar:
                for future in as_completed(future_to_index):
                    index = future_to_index[future]
                    try:
                        result = future.result()
                    except Exception as exc:
                        print(f'Generated an exception: {exc}')
                        result = None
                    results.append((index, result))
                    pbar.update(1)

        # Sort results by index and extract only the values
        sorted_results = [r[1] for r in sorted(results, key=lambda x: x[0])]
        
        return sorted_results

    if __name__ == '__main__':
        # Process the geometries
        results = process_geometries(combined_gdf)

        # Add the results as a new column to the GeoDataFrame
        combined_gdf['mean_agb'] = results

        # Save as GeoJSON
        combined_gdf[['geometry', 'mean_agb']].to_file("agb.geojson", driver="GeoJSON")


Model already exists


### Tune hyperparameters for above ground biomass regressor using Optuna

In [5]:
import pandas as pd
import numpy as np
import optuna
import tensorflow as tf
import joblib
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.optimizers import Adam

class NeuralNetwork:
    def __init__(self, input_shape, layers, dropout_rate, learning_rate, device):
        self.input_shape = input_shape
        self.layers = layers
        self.dropout_rate = dropout_rate
        self.learning_rate = learning_rate
        self.device = device
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Input(shape=(self.input_shape,)))
        for layer_size in self.layers:
            model.add(Dense(layer_size, activation='relu'))
            model.add(Dropout(self.dropout_rate))
        model.add(Dense(1, activation='linear'))  # Output layer for regression
        return model

    def compile_model(self):
        optimizer = Adam(learning_rate=self.learning_rate)
        self.model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])  # Use MSE for regression

    def train_model(self, X_train, y_train, epochs=20, batch_size=32, validation_split=0.2):
        with tf.device(self.device):
            history = self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
        return history

    def evaluate_model(self, X_test, y_test):
        with tf.device(self.device):
            loss, mae = self.model.evaluate(X_test, y_test)
        return loss, mae

    def predict(self, X_test):
        with tf.device(self.device):
            predictions = self.model.predict(X_test)
        return predictions.flatten()

    def calculate_rmse(self, y_test, predictions):
        return np.sqrt(mean_squared_error(y_test, predictions))
    
    def save_model(self, filename):
        self.model.save(filename)

    @classmethod
    def load_model(cls, filename, input_shape, device):
        loaded_model = tf.keras.models.load_model(filename)
        nn = cls(input_shape, [], 0, 0, device)  # Dummy values for layers, dropout_rate, and learning_rate
        nn.model = loaded_model
        return nn

def objective(trial):
    layers = []
    for i in range(trial.suggest_int('n_layers', 1, 3)):
        layers.append(trial.suggest_int(f'n_units_l{i}', 64, 512))
    
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)
    
    nn = NeuralNetwork(input_shape=X_train.shape[1], device=device, layers=layers, dropout_rate=dropout_rate, learning_rate=learning_rate)
    nn.compile_model()
    
    nn.train_model(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)
    
    predictions = nn.predict(X_test)
    predictions_exp = np.exp(predictions)  # Inverse log transform predictions
    rmse = nn.calculate_rmse(y_test_exp, predictions_exp)
    
    return rmse

if os.path.exists('models/agb_regression_model.h5'):
    print("Model already exists")
else:
    # Assuming combined_gdf is already loaded
    combined_gdf_filtered = combined_gdf.dropna()
    X = combined_gdf_filtered['embeddings'].tolist()
    X = pd.DataFrame(X)
    y = combined_gdf_filtered['mean_agb']

    # Log transform the target variable
    y_log = np.log(y + 1)  # Adding 1 to avoid log(0)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)

    # Scale the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Inverse transform for evaluation
    y_test_exp = np.exp(y_test) - 1  # Subtract 1 to get the original scale

    device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'

    # Optimize the hyperparameters
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=5)

    # Print the best hyperparameters
    print(study.best_params)

    # Example usage with the best hyperparameters
    best_params = study.best_params
    layers = [best_params[f'n_units_l{i}'] for i in range(best_params['n_layers'])]
    dropout_rate = best_params['dropout_rate']
    learning_rate = best_params['learning_rate']

    nn = NeuralNetwork(input_shape=X_train.shape[1], device=device, layers=layers, dropout_rate=dropout_rate, learning_rate=learning_rate)
    nn.compile_model()
    history = nn.train_model(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)
    loss, mae = nn.evaluate_model(X_test, y_test)
    print(f'Test MAE: {mae:.4f}')
    predictions = nn.predict(X_test)
    predictions_exp = np.exp(predictions) - 1  # Inverse log transform
    rmse = nn.calculate_rmse(y_test_exp, predictions_exp)
    print(f'Test RMSE: {rmse:.4f}')

    # Save the model
    nn.save_model('models/agb_regression_model.h5')

    # Save scaler
    joblib.dump(scaler, 'models/3_scaler.joblib')


/opt/conda/envs/claymodel-latest-v2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-26 15:29:41.264463: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 15:29:41.530251: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 15:29:41.603764: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-26 15:29:42.194784: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlo

Model already exists


In [18]:
combined_gdf_filtered

,embeddings,geometry,mean_agb
0,"[-0.1340062, 0.077923074, -0.0016371261, 0.083...","POLYGON ((-116.13800 34.50819, -116.13800 34.5...",0.336093
1,"[-0.12408799, 0.06986915, -0.0014471954, 0.080...","POLYGON ((-116.15667 34.50819, -116.15667 34.5...",0.315462
2,"[-0.13270076, 0.067874126, -0.0058731856, 0.08...","POLYGON ((-116.13800 34.48952, -116.13800 34.5...",0.308832
3,"[-0.11633471, 0.07077362, -0.009755048, 0.0875...","POLYGON ((-116.13800 34.47086, -116.13800 34.4...",0.263030
4,"[-0.1335075, 0.0719392, -0.0011160049, 0.08034...","POLYGON ((-116.11934 34.48952, -116.11934 34.5...",0.281872
...,...,...,...
125141,"[-0.09712396, 0.06625347, -0.056792237, 0.1275...","POLYGON ((-123.32467 40.51886, -123.32467 40.5...",95.744820
125142,"[-0.09359632, 0.06506389, -0.053921893, 0.1259...","POLYGON ((-123.34334 40.53752, -123.34334 40.5...",90.395423
125143,"[-0.093709834, 0.06074024, -0.07075437, 0.1278...","POLYGON ((-123.23134 39.32419, -123.23134 39.3...",95.926339
125144,"[-0.09378435, 0.059857186, -0.055076838, 0.128...","POLYGON ((-123.45534 39.10019, -123.45534 39.1...",77.902087


In [1]:
import datetime
import glob
import math
import os
import random
import requests

import geopandas as gpd
import lancedb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pystac_client
import shapely
from shapely.geometry import box, Polygon
import torch
import yaml
from box import Box
from pyproj import Transformer
from rasterio.io import MemoryFile
from stacchip.chipper import Chipper
from stacchip.indexer import Sentinel2Indexer
from stacchip.processors.prechip import normalize_timestamp
from torchvision.transforms import v2

from src.model import ClayMAEModule

/opt/conda/envs/claymodel-latest-v2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/claymodel-latest-v2/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:

# Optimize GDAL settings for cloud optimized reading
os.environ["GDAL_DISABLE_READDIR_ON_OPEN"] = "EMPTY_DIR"
os.environ["AWS_REQUEST_PAYER"] = "requester"

STAC_API = "https://earth-search.aws.element84.com/v1"
COLLECTION = "sentinel-2-l2a"
YEAR = 2022

# Search the catalogue
catalog = pystac_client.Client.open(STAC_API)
search = catalog.search(
    collections=[COLLECTION],
    datetime=f"{YEAR}-01-01T00:00:00Z/{YEAR+1}-01-01T00:00:00Z",
    bbox=[-123.30442043188332, 46.875515943543945, -121.65107564263492, 48.012480708758716],
    max_items=100,
    query={"eo:cloud_cover": {"lt": 80}},
)

all_items = search.get_all_items()

# Reduce to one per date (there might be some duplicates
# based on the location)
items = []
dates = []
for item in all_items:
    if item.datetime.date() not in dates:
        items.append(item)
        dates.append(item.datetime.date())

print(f"Found {len(items)} items")

/opt/conda/envs/claymodel-latest-v2/lib/python3.11/site-packages/pystac_client/item_search.py:851: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


Found 23 items


In [3]:
chips = []
datetimes = []
bboxs = []
chip_ids = []
item_ids = []

for item in items:
    print(f"Working on {item}")

    # Index the chips in the item
    indexer = Sentinel2Indexer(item)

    # Instanciate the chipper
    chipper = Chipper(indexer, assets=["red", "green", "blue", "nir", "scl"])

    # Get first chip for the "image" asset key
    for idx, (x, y, chip) in enumerate(chipper):
        if idx > 2:
            break
        del chip["scl"]
        chips.append(chip)
        datetimes.append(item.datetime)
        bboxs.append(indexer.get_chip_bbox(x, y))
        chip_ids.append((x, y))
        item_ids.append(item.id)

Working on <Item id=S2B_10TDS_20221231_0_L2A>
Working on <Item id=S2B_10TDT_20221228_0_L2A>
Working on <Item id=S2A_10TFS_20221226_0_L2A>
Working on <Item id=S2B_10TDS_20221221_0_L2A>
Working on <Item id=S2B_10TFS_20221218_0_L2A>
Working on <Item id=S2A_10TDS_20221216_0_L2A>
Working on <Item id=S2A_10TES_20221213_0_L2A>
Working on <Item id=S2B_10TDT_20221211_0_L2A>
Working on <Item id=S2A_10TFS_20221206_0_L2A>
Working on <Item id=S2A_10TES_20221203_0_L2A>
Working on <Item id=S2B_10TDT_20221201_0_L2A>
Working on <Item id=S2B_10TDS_20221128_0_L2A>
Working on <Item id=S2A_10TDS_20221126_0_L2A>
Working on <Item id=S2A_10TDS_20221123_0_L2A>
Working on <Item id=S2B_10TDS_20221121_0_L2A>
Working on <Item id=S2B_10TDS_20221118_0_L2A>
Working on <Item id=S2A_10TDS_20221116_0_L2A>
Working on <Item id=S2A_10TFS_20221113_0_L2A>
Working on <Item id=S2B_10TFS_20221111_0_L2A>
Working on <Item id=S2B_10TDS_20221108_0_L2A>
Working on <Item id=S2A_10TDT_20221106_0_L2A>
Working on <Item id=S2B_10TFS_2022

In [4]:
pixels = np.array([np.array(list(chip.values())).squeeze() for chip in chips])
pixels.shape

(69, 4, 224, 224)

In [ ]:
# Extract mean, std, and wavelengths from metadata
platform = "sentinel-2-l2a"
# Retrieve the file content from the URL

url = (
    "https://raw.githubusercontent.com/Clay-foundation/model/main/configs/metadata.yaml"
)
response = requests.get(url, allow_redirects=True)

# Convert bytes to string
content = response.content.decode("utf-8")

# Load the yaml
content = yaml.safe_load(content)

metadata = Box(content)
mean = []
std = []
waves = []
# Use the band names to get the correct values in the correct order.
for band in chips[0].keys():
    mean.append(metadata[platform].bands.mean[band])
    std.append(metadata[platform].bands.std[band])
    waves.append(metadata[platform].bands.wavelength[band])

# Prepare the normalization transform function using the mean and std values.
transform = v2.Compose(
    [
        v2.Normalize(mean=mean, std=std),
    ]
)